# 데이터 준비하기 : 조인, 병합, 변형

### 1. 계층적 색인

In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
data=pd.Series(np.random.randn(9),
               index=[['a','a','a','b','b','c','c','d','d'],
                      [1, 2,    3,  1,  3,  1,  2,  2,  3]])

In [4]:
data

a  1    0.290263
   2   -1.587549
   3    0.323646
b  1    1.020193
   3    0.857158
c  1    1.122084
   2   -0.070879
d  2    0.883866
   3   -0.165739
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
data['b']

1    1.020193
3    0.857158
dtype: float64

In [7]:
data['b':'c']

b  1    1.020193
   3    0.857158
c  1    1.122084
   2   -0.070879
dtype: float64

In [8]:
data.loc[['b','d']]

b  1    1.020193
   3    0.857158
d  2    0.883866
   3   -0.165739
dtype: float64

In [9]:
data.loc[:,2]

a   -1.587549
c   -0.070879
d    0.883866
dtype: float64

In [10]:
data.unstack()

,1,2,3
a,0.290263,-1.587549,0.323646
b,1.020193,NaN,0.857158
c,1.122084,-0.070879,NaN
d,NaN,0.883866,-0.165739


In [11]:
data.unstack().stack()

a  1    0.290263
   2   -1.587549
   3    0.323646
b  1    1.020193
   3    0.857158
c  1    1.122084
   2   -0.070879
d  2    0.883866
   3   -0.165739
dtype: float64

In [12]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),
                  index=[['a','a','b','b'],[1,2,1,2]],
                  columns=[['Ohio','Ohio','Colorado'],
                          ['Green','Red','Green']])

In [13]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [14]:
frame.index.names=['key1','key2']

In [15]:
frame.columns.names=['state','color']

In [16]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [20]:
frame['state']

KeyError: 'state'

In [ ]:
frame.swaplevel('key1','key2')

In [ ]:
frame.sort_index(level=1)

In [ ]:
frame.swaplevel(0,1).sort_index(level=0)

In [ ]:
frame.sum(level='key2')

In [ ]:
frame.sum(level='color',axis=1)

In [ ]:
frame=pd.DataFrame({'a':range(7),'b':range(7,0,-1),
                   'c':['one','one','one','two','two','two','two'],
                    'd':[0,1,2,0,1,2,3]})

In [ ]:
frame

In [ ]:
frame2=frame.set_index(['c','d'])

In [ ]:
frame2

In [ ]:
frame.set_index(['c','d'],drop=False)

In [ ]:
frame2.reset_index()

### 데이터 합치기

- pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터베이스의 join 연산과 유사하다.
- pandas.concat은 하나의 축을 따라 객체를 이어붙인다.
- combile_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

In [ ]:
df1=pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                 'data1':range(7)})

In [ ]:
df2=pd.DataFrame({'key':['a','b','d'],
                 'data2':range(3)})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1,df2)

In [ ]:
pd.merge(df1,df2,on='key')

In [ ]:
df3=pd.DataFrame({'1key':['b','b','a','c','a','a','b'],
                 'data1':range(7)})

In [ ]:
df4=pd.DataFrame({'rkey':['a','b','d'],
                 'data2':range(3)})

In [ ]:
pd.merge(df3,df4,left_on='1key',right_on='rkey')

### 색인 병합하기

In [ ]:
left1=pd.DataFrame({'key':['a','b','a','a','b','c'],
                   'value':range(6)})

In [ ]:
right1=pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

In [ ]:
left1

In [ ]:
right1

In [ ]:
pd.merge(left1,right1,left_on='key',right_index=True)

### 겹치는 데이터 합치기

In [22]:
a=pd.Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],
           index=['f','e','d','c','b','a'])

In [23]:
b=pd.Series(np.arange(len(a),dtype=np.float64),
           index=['f','e','d','c','b','a'])

In [24]:
b[-1]=np.nan

In [25]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [26]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [27]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [28]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [29]:
df1=pd.DataFrame({'a':[1.,np.nan,5.,np.nan],
                 'b':[np.nan,2.,np.nan,6.],
                 'c':range(2,18,4)})

In [30]:
df2=pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                 'b':[np.nan,3.,4.,6.,8.]})

In [31]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [32]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [33]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 재형성과 피벗

- stack : 데이터의 컬럼을 로우로 피벗(또는 회전)시킨다.
- unstack : 로우를 컬럼으로 피벗시킨다. 

In [34]:
data=pd.DataFrame(np.arange(6).reshape((2,3)),
                 index=pd.Index(['Ohio','Colorado'],name='state'),
                 columns=pd.Index(['one','two','three'],
                                 name='number'))

In [35]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [36]:
result=data.stack()

In [37]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [38]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [39]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [41]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [42]:
s1=pd.Series([0,1,2,3],index=['a','b','c','d'])

In [43]:
s2=pd.Series([4,5,6],index=['c','d','e'])

In [44]:
data2=pd.concat([s1,s2],keys=['one','two'])

In [45]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [46]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [47]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [48]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [50]:
df=pd.DataFrame({'left':result,'right':result + 5},
               columns=pd.Index(['left','right'],name='side'))

In [51]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [52]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [53]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [74]:
data=pd.read_csv('../examples/macrodata.csv')

In [75]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [70]:
periods=pd.PeriodIndex(year=data.year,quarter=data.quarter,name='date')

In [71]:
columns=pd.Index(['realgdp','infl','unemp'],name='item')

In [79]:
data.reindex(columns=columns)

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [85]:
data.index=periods.to_timestamp('D','end')

In [62]:
ldata=data.stack(0).reset_index().rename(columns={0:'value'})

In [63]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [65]:
pivoted=ldata.pivot('date','item','value')

In [66]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [87]:
ldata['value2']=np.random.randn(len(ldata))

In [88]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.497795
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.514573
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.213994
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-1.778233
4,1959-06-30 23:59:59.999999999,infl,2.340,1.756565
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.478515
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.034113
7,1959-09-30 23:59:59.999999999,infl,2.740,1.187341
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.389068
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.439859


In [90]:
pivoted=ldata.pivot('date','item')

In [91]:
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.514573 -1.497795   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.756565 -1.778233   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.187341  0.034113   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.967195 -0.439859   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  1.230851  0.711671   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.213994  
1959-06-30 23:59:59.999999999  0.478515  
1959-09-30 23:59:59.999999999  0.389068  
1959-12-31 23:59:59.999999999  1.264469  
1960-03-31 23:59:59.999999999  0.176312

In [92]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [94]:
unstacked=ldata.set_index(['date','item']).unstack('item')

In [95]:
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.514573 -1.497795   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.756565 -1.778233   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  1.187341  0.034113   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.967195 -0.439859   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  1.230851  0.711671   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.706701  0.566861   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.902782 -1.329483   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.213994  
1959-06-30 23:59:59.999999999  0.478515  
1959-09-30 23:59:59.999999999  0.389068  
1959-12-31 23:59:59.999999999  1.264469  
1960-03-31 23:59:59.999999999  0.176312  
1960-06-30 23:59:59.999999999  0.971137  
1960-09-30 23:59:59.999999999  0.458218

### 넓은 형식에서 긴 형식으로 피벗하기

In [97]:
df=pd.DataFrame({'key':['foo','bar','baz'],
                'A':[1,2,3],
                'B':[4,5,6],
                'C':[7,8,9]})

In [98]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [99]:
melted=pd.melt(df,['key'])

In [100]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [101]:
reshaped=melted.pivot('key','variable','value')

In [102]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [103]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [106]:
pd.melt(df,id_vars=['key'],value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [107]:
pd.melt(df,value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [108]:
pd.melt(df,value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
